<a href="https://colab.research.google.com/github/nguyenleanhquan2005/Project_SER/blob/main/B%E1%BA%A3n_sao_c%E1%BB%A7a_RAG_with_LangChain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -qU \
      python-dotenv \
      langchain \
      langchain_openai \
      langchain_community \
      langchain-huggingface \
      langchain-google-genai \
      streamlit \
      faiss-cpu \
      sentence-transformers \
      pypdf \
      docx2txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 90.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 51.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.5/310.5 kB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 61.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.5/447.5 kB 29.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 92.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source o

In [ ]:
#Khai báo khóa API key
from google.colab import userdata
import os
os.environ["GOOGLE_API_KEY"]  = userdata.get('GOOGLE_API_KEY')

# Xây dựng chatbot đơn giãn với RAG

## Xây dựng Vector database

In [ ]:
#Tải dữ liệu từ folder (.txt)
from langchain_community.document_loaders import DirectoryLoader, PyPDFLoader
file_path = "/content/Tom_tat_kien_thuc_NLP_quan_trong.pdf"
loader = PyPDFLoader(file_path)
docs = loader.load()

In [ ]:
print(docs[0].page_content[:200])

Tóm tắt Kiến thức NLP Quan trọng
1. Regular expressions, Text normalization, Edit distance
- a. Regular Expressions: tìm kiếm và trích xuất mẫu từ văn bản.
- b. Text Normalization: chuẩn hóa văn bản (


In [ ]:
from langchain_text_splitters.character import  RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings

text_splitter = RecursiveCharacterTextSplitter(
      chunk_size=400,
      chunk_overlap=200
)
doc_chunks = text_splitter.split_documents(docs)
#Embedding các chunks này và đưa vào vector store
embeddings = HuggingFaceEmbeddings()
vectordb = FAISS.from_documents(doc_chunks, embeddings)

In [ ]:
# Lưu vector store
vectordb.save_local("index")

## Xây dựng chain

In [ ]:
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash")
retriever = vectordb.as_retriever(
        search_kwargs={
                  "k": 2, # sl kq truy vấn trả về
    }
    )
memory = ConversationBufferMemory(
        llm=llm,
        output_key="answer",
        memory_key="chat_history",
        return_messages=True
    )
chain = ConversationalRetrievalChain.from_llm(
        llm=llm,
        retriever=retriever,
        chain_type="map_reduce",
        memory=memory,
        verbose=True
    )

In [ ]:
memory.clear()
while True:
    user_input = input("user: ")
    if user_input.lower() == "exit":
        break

    response = chain({"question": user_input, "chat_history": memory.chat_memory.messages})
    assistant_response = response["answer"]

    print("assistant: ", assistant_response)

user: kiến thức quan trọng của bài là gì


> Entering new MapReduceDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following portion of a long document to see if any of the text is relevant to answer the question.
Return any relevant text verbatim.
______________________
Tóm tắt Kiến thức NLP Quan trọng
1. Regular expressions, Text normalization, Edit distance
- a. Regular Expressions: tìm kiếm và trích xuất mẫu từ văn bản.
- b. Text Normalization: chuẩn hóa văn bản (lowercase, stemming, lemmatization).
- c. Edit Distance (Levenshtein): đo khoảng cách giữa hai chuỗi.
2. N-Gram & Language Models
- a. N-grams: mô hình xác suất dựa trên chuỗi từ.
Human: kiến thức quan trọng của bài là gì
Prompt after formatting:
System: Use the following portion of a long document to see if any of the text is relevant to answer the question.
Return any relevant text verbatim.
______________________
- c. Edit Distance (Levenshtein): đo khoảng cách giữa ha

KeyboardInterrupt: Interrupted by user

In [ ]:
memory.chat_memory.messages